# Dependencies

- math module

# Traditional Bond

$$
PV = - (C[\frac{1 - \frac{1}{(1+y)^N}}{y}] + \frac{FV}{(1+y)^N})
$$

## Yield to Maturity (YTM)

Using the Newton-Raphson method: 
$$y_{n+1} = y_n - \frac{F(y_n)}{F'(y_n)}$$
where 
$$ F(y) = \frac{C}{1+y} + \frac{C}{(1+y)^2} + ... + \frac{C}{(1+y)^N} + \frac{FV}{(1+y)^N} + PV \\ = C[ \frac{1 - \frac{1}{(1 + y)^N} }{y}] + \frac{FV}{(1+y)^N} + PV $$
and
$$ F'(y) = -\frac{C}{(1+y)^2} - \frac{2C}{(1+y)^3} - ... - \frac{N * C}{(1 + y)^{N+1}} - \frac {N * FV}{(1+y)^{N+1}} \\ = -C \{ \frac{1}{y^2}[1 - \frac{1}{(1 + y)^N}] - \frac{N}{y (1 + y)^{N+1}} \}$$

In [1]:
def F(y, c, fv, n, pv):
    return c * ( (1 - 1 / ( (1 + y) ** n ) ) / y ) + fv / ( (1 + y) ** n ) + pv

In [2]:
def dF(y, c, fv, n, pv):
    return -c * ( (1 / y ** 2) * (1 -  1 / (1 + y) ** n ) ) - n / (y * (1 + y) ** (n + 1))

In [3]:
def YTM(c, fv, n, pv, itr=1000):
    """
        c: periodic coupon rate, or payment
        fv: future value of the series of cash flows
        n: number of periods
        pv: present value of the cash flow
        
        The resulting YTM is a periodic rate.
    """
    y = c / fv
    for i in range(itr):
        y = y - F(y, c, fv, n, pv) / dF(y, c, fv, n, pv)
    return y * 100

In [4]:
YTM(c=10, fv=1000, n=10, pv=-900)

2.1202829427100416

## Present Value (PV) and Future Value (FV)

In [5]:
def PV(c, fv, n, y):
    y /= 100
    return - (c * (1 - 1 / (1+y) ** n) / y + fv / (1+y) ** n)

In [6]:
def FV(c, pv, n, y):
    y /= 100
    return (1 + y) ** n * (-pv - c / y) + c / y

In [7]:
PV(10, 1000, 10, 2.1202829427100416)

-899.9999999999957

In [8]:
FV(10, -900, 10, 2.1202829427100416)

1000.0000000000053

## Number of Periods (N)

$$
N = \frac{ln(\frac{FV - C/y}{-PV - C/y})}{ln(1+y)}
$$

In [9]:
import math

In [10]:
def N(c, pv, fv, y):
    y /= 100
    return math.log((fv - c/y) / (-pv - c/y)) / math.log(1+y)

In [11]:
N(10, -900, 1000, 2.1202829427100416)

9.999999999999515

# Bootstrap Pricing

Suppose we are given a list specifying YTMs of bonds with different maturities, and the same face value.  Based on the info, we want to construct a list of spot rates, as well as a list of one-period forward rates.

Some constraints on the given list:
- We assume semi-annual periods;
- The list should have values at the start and the end, otherwise we cannot apply linear interpolation;
- 'None' implies that interpolation should be applied to the entry where the object is located.

Other notes:
- We can, with no doubt, make use of numpy in linear interpolation, but that would be unnecessary for lightweight usage.
- Assume without loss of generality that Par = $1,000.  In fact, whatever par we use, the result will be the same.

In [12]:
def linear_interpolation(ys):
    if len(ys) == 0:
        raise Exception('An empty list is provided')
    if ys[0] is None or ys[-1] is None:
        raise Exception('Values must be provided at two ends of the list')
    start = -1
    while start < len(ys):
        while start < len(ys)-1 and ys[start+1] is not None:
            start += 1
        if start == len(ys)-1:
            break
        
        end = start + 1
        while end < len(ys) and ys[end] is None:
            end += 1
        if end == len(ys):
            break
        # fill in the points between start and end
        delta = (ys[end] - ys[start]) / (end - start)
        for i in range(end - start - 1):
            ys[start+i+1] = ys[start+i] + delta
        # continue
        start = end
    return ys

In [13]:
ys_test = [None] * 10

In [14]:
ys_test[0] = 8.
ys_test[9] = 5.75

In [15]:
linear_interpolation(ys_test)

[8.0, 7.75, 7.5, 7.25, 7.0, 6.75, 6.5, 6.25, 6.0, 5.75]

In [16]:
srs = [None] * len(ys_test)

In [17]:
def spot_rates(ys, srs):
    len_ys = len(ys)
    if len_ys < 3:
        return ys
    srs = [None] * len_ys
    srs[0:2] = ys[0:2]
    for i in range(2, len_ys):
        if srs[i] is None:
            c = 1000 * ys[i] / 200
            coupon_pv_total = 0
            for j in range(i):
                coupon_pv_total += c / (1 + srs[j] / 200) ** (j+1)
            current_sr = (((1000 + c) / (1000 - coupon_pv_total)) ** (1/(i+1)) - 1) * 200
            srs[i] = current_sr
    return srs

In [18]:
srs = spot_rates(ys_test, srs)

In [19]:
srs

[8.0,
 7.75,
 7.487294045722326,
 7.226940325883424,
 6.964229195868654,
 6.69948578849211,
 6.43305157893983,
 6.1652835671370365,
 5.896551491608948,
 5.6272342692710975]

In [20]:
def fwd_rates(srs):
    """
        Given a list of spot rates, compute one-period forward rate
    """
    len_srs = len(srs)
    if len_srs == 0:
        raise Exception("The spot rates list is empty")
    frs = [None] * len_srs
    frs[0] = srs[0]
    current_compound_factor = 1 + frs[0] / 200
    for i in range(1, len_srs):
        current_fwd_rate = ((1 + srs[i] / 200) ** (i+1) / (1 + srs[i-1] / 200) ** i - 1) * 200
        frs[i] = current_fwd_rate
    return frs

In [21]:
fwd_rates(srs)

[8.0,
 7.500300480769262,
 6.962878315190935,
 6.447837668188194,
 5.9167109661543815,
 5.380839892049316,
 4.84164291671112,
 4.300607443326365,
 3.7592669248612154,
 3.219176379446065]

In [22]:
srs_rounded = [round(r, 2) for r in srs]

In [23]:
srs_rounded

[8.0, 7.75, 7.49, 7.23, 6.96, 6.7, 6.43, 6.17, 5.9, 5.63]

In [24]:
def price_with_spot_rates(srs, c, par):
    n = len(srs)
    price = 0
    c /= 2
    for i in range(n-1):
        price += c / (1 + srs[i]/200) ** (i+1)
        print(1 + srs[i]/200, i+1)
    price += (par + c) / (1 + srs[n-1]/200) ** n
    return price

In [25]:
price_with_spot_rates(srs_rounded[0:8], 6, 100)

1.04 1
1.03875 2
1.03745 3
1.03615 4
1.0348 5
1.0335 6
1.03215 7


99.12286907270496